In [1]:
#ipynb config
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn.cross_validation import train_test_split
from models.cascade_base import CascadeBase
from utils.visualizer import Visualizer
from IPython.display import SVG, display

import numpy as np
import matplotlib.pyplot as plt

import time
import theano
import lasagne

plt.style.use('ggplot')

%matplotlib inline

In [3]:
network = CascadeBase(img_shape=(512, 512),
                      learning_rate=1e-3,
                      c=1e-1,
                      c_complexity=5e-1,
                      c_sub_objs=[5e-1, 1, 1],
                      c_sub_obj_cs=[1e-2, 1e-1, 1e-1],
                      mul=True,
                      pool_sizes=[2, 2, 2, 2],
                      num_filters=[1, 1, 3, 6],
                      filter_sizes=[1, 3, 3, 3],
                      l2_c=1e-4)

In [4]:
network.load('../weights/', '3cascades_stable_recall')

In [5]:
max_pool = theano.function([network.targets], lasagne.layers.get_output(network.target_pool_layers[-1]))

In [75]:
step = 512 / 32

with open('../data/info.dat', 'wb') as f:
    for k in range(5):
        images = np.load('../data/imgs_000{}.npy'.format(k))
        masks = np.load('../data/masks_000{}.npy'.format(k))
        max_pooled = max_pool(masks.reshape((-1, 1, 512, 512))).reshape(-1, 32, 32)

        for i, mask in enumerate(max_pooled):
            f.write('pos{}_{}.png'.format(k, i))
            plt.imsave('../data/pos{}_{}.png'.format(k, i), images[i])
            num_pos = 0
            bounding_rectangles = []

            x, y = mask.nonzero()

            for j in range(len(x)):
                bounding_rectangles.append([x[j] * step, y[j] * step, step, step])

            f.write('\t{}'.format(len(bounding_rectangles)))
            for br in bounding_rectangles:
                f.write('\t')
                f.write(' '.join([str(e) for e in br]))
            f.write('\n')

In [6]:
bg_images = np.load('../data/bg_imgs_0000.npy')

In [7]:
with open('../data/bg.txt', 'wb') as f:
    for i, bg in enumerate(bg_images):
        f.write('./bg0_{}.png\n'.format(i))
        plt.imsave('../data/bg0_{}.png'.format(i), bg)